<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [233]:
# Import statements
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [234]:
data = 'https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv'

In [235]:
# reading data as csv

df = pd.read_csv(data)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [236]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [237]:
df['TotalCharges'].value_counts()

20.2      11
          11
19.75      9
19.65      8
20.05      8
          ..
4399.5     1
1421.9     1
266.8      1
1821.8     1
223.9      1
Name: TotalCharges, Length: 6531, dtype: int64

In [238]:
df['TotalCharges'] = df['TotalCharges'].replace(' ', )
df['TotalCharges'].value_counts()

20.2       11
19.75       9
20.05       8
19.65       8
19.9        8
           ..
5763.3      1
4541.2      1
4374.55     1
229.4       1
223.9       1
Name: TotalCharges, Length: 6530, dtype: int64

In [239]:
df['TotalCharges'] = df['TotalCharges'].astype(float)

In [240]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [241]:
df['TotalCharges'].describe() 

count    7043.000000
mean     2285.169963
std      2267.544511
min        18.800000
25%       401.675000
50%      1398.250000
75%      3803.050000
max      8684.800000
Name: TotalCharges, dtype: float64

In [242]:
df = df.drop('customerID', axis=1)
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2285.169963
std,0.368612,24.559481,30.090047,2267.544511
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.675000
50%,0.000000,29.000000,70.350000,1398.250000
75%,0.000000,55.000000,89.850000,3803.050000
max,1.000000,72.000000,118.750000,8684.800000


In [243]:
test = df['Dependents']
test.value_counts()

No     4933
Yes    2110
Name: Dependents, dtype: int64

In [258]:
# Do not do this, this is brute force coding.  With more time I should be able to write a function
# that would encode the categories appropriately, or use a better encoding method.  Pressed for time!!!

df['gender'] = df['gender'].replace('Female', 1).replace('Male', 0)
df['Partner'] = df['Partner'].replace('Yes', 1).replace('No', 0)
df['Dependents'] = df['Dependents'].replace('Yes', 1).replace('No', 0)
df['PhoneService'] = df['PhoneService'].replace('Yes', 1).replace('No', 0)
df['MultipleLines'] = df['MultipleLines'].replace('Yes', 1).replace('No', 0).replace('No phone service', 2)
df['InternetService'] = df['InternetService'].replace('DSL', 1).replace('No', 0).replace('Fiber optic', 2)
df['OnlineSecurity'] = df['OnlineSecurity'].replace('Yes', 1).replace('No', 0).replace('No internet service', 2)
df['OnlineBackup'] = df['OnlineBackup'].replace('Yes', 1).replace('No', 0).replace('No internet service', 2)
df['DeviceProtection'] = df['DeviceProtection'].replace('Yes', 1).replace('No', 0).replace('No internet service', 2)
df['TechSupport'] = df['TechSupport'].replace('Yes', 1).replace('No', 0).replace('No internet service', 2)
df['StreamingTV'] = df['StreamingTV'].replace('Yes', 1).replace('No', 0).replace('No internet service', 2)
df['StreamingMovies'] = df['StreamingMovies'].replace('Yes', 1).replace('No', 0).replace('No internet service', 2)
df['Contract'] = df['Contract'].replace('One year', 1).replace('Month-to-month', 0).replace('Two year', 2)
df['PaperlessBilling'] = df['PaperlessBilling'].replace('Yes', 1).replace('No', 0)
df['PaymentMethod'] = df['PaymentMethod'].replace('Electronic check', 0).replace('Mailed check', 1).replace('Bank transfer (automatic)', 2).replace('Credit card (automatic)', 3)
df['Churn'] = df['Churn'].replace('Yes', 1).replace('No', 0)

df.head(3)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,0.023682,0,2,1,0,1,0,0,0,0,0,1,0,0.706908,0.706908,0
1,0,0,0,0,0.017983,1,0,1,1,0,1,0,0,0,1,0,1,0.030122,0.999384,0
2,0,0,0,0,0.016552,1,0,1,1,1,0,0,0,0,0,1,1,0.445662,0.895048,1


In [272]:
df.dtypes

gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure              float64
PhoneService          int64
MultipleLines         int64
InternetService       int64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract              int64
PaperlessBilling      int64
PaymentMethod         int64
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

In [261]:
# columns to encode
     
encoded = ['gender', 
            'Partner', 
            'Dependents', 
            'PhoneService', 
            'MultipleLines', 
            'InternetService', 
            'OnlineSecurity',
            'OnlineBackup',
            'DeviceProtection',
            'TechSupport',
            'StreamingTV',
            'StreamingMovies',
            'Contract',
            'PaperlessBilling',
            'PaymentMethod',
            'Churn']

In [274]:
X = df.drop("Churn", axis='columns').values
y = df.Churn.values

In [280]:
# Test/Train split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [281]:
print(X_train.shape,
      y_train.shape,
      X_test.shape,
      y_test.shape)

(5634, 19) (5634,) (1409, 19) (1409,)


In [282]:
# normalizing values in selected columns (could normalize all data, not sure which approach to take)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [285]:
# Important Hyperparameters
inputs = X_train.shape[1]
epochs = 10
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile Model
# model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Fit Model
model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Train on 5634 samples, validate on 1409 samples
Epoch 1/10
5634/5634 [==============================] - 1s 216us/sample - loss: 0.1645 - mse: 0.1645 - mae: 0.3110 - val_loss: 0.1417 - val_mse: 0.1417 - val_mae: 0.2907
Epoch 2/10
5634/5634 [==============================] - 1s 140us/sample - loss: 0.1408 - mse: 0.1408 - mae: 0.2847 - val_loss: 0.1440 - val_mse: 0.1440 - val_mae: 0.2842
Epoch 3/10
5634/5634 [==============================] - 1s 133us/sample - loss: 0.1375 - mse: 0.1375 - mae: 0.2788 - val_loss: 0.1414 - val_mse: 0.1414 - val_mae: 0.2754
Epoch 4/10
5634/5634 [==============================] - 1s 127us/sample - loss: 0.1347 - mse: 0.1347 - mae: 0.2736 - val_loss: 0.1448 - val_mse: 0.1448 - val_mae: 0.2782
Epoch 5/10
5634/5634 [==============================] - 1s 126us/sample - loss: 0.1329 - mse: 0.1329 - mae: 0.2713 - val_loss: 0.1432 - val_mse: 0.1432 - val_mae: 0.2726
Epoch 6/10
5634/5634 [==============================] - 1s 153us/sample - loss: 0.1315 - mse: 0.1315 -

In [288]:


def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=19, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0) # we pass function created above to our KerasClass
                                                          # works as any other model would 

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],    # these are some of the params we will be manipulating
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)  # Not dist. training, dist exp trying to
grid_result = grid.fit(X_train, y_train)                                          

# Note: look into what is cross validate, again!

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7971262574195862 using {'batch_size': 60, 'epochs': 20}
Means: 0.795884644985199, Stdev: 0.00787608168179345 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7910909533500672, Stdev: 0.003972859821958846 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7941090703010559, Stdev: 0.0070466372324608315 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7971262574195862, Stdev: 0.005129949055509983 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7923349142074585, Stdev: 0.012431704683379969 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7948192358016968, Stdev: 0.011981787785375463 with: {'batch_size': 100, 'epochs': 20}


In [289]:
# define the grid search parameters
param_grid = {'batch_size': [20],
              'epochs': [20, 40, 60,200]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7948183059692383 using {'batch_size': 20, 'epochs': 200}
Means: 0.793044137954712, Stdev: 0.008548088966559899 with: {'batch_size': 20, 'epochs': 20}
Means: 0.792157781124115, Stdev: 0.012266078785684306 with: {'batch_size': 20, 'epochs': 40}
Means: 0.7933985829353333, Stdev: 0.0059830265954845556 with: {'batch_size': 20, 'epochs': 60}
Means: 0.7948183059692383, Stdev: 0.006495783781343724 with: {'batch_size': 20, 'epochs': 200}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?